In [ ]:
# Needed packages to make this codebase working
!pip install numpy matplotlib

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os   # Create a directory to save images if it doesn't exist

"""
The vector components that represent the fractal
More information can be found here: https://en.wikipedia.org/wiki/Barnsley_fern
"""
def function1(x, y):
    return (0, 0.16 * y)

def function2(x, y):
    return (0.85 * x + 0.04 * y, -0.04 * x + 0.85 * y + 1.6)

def function3(x, y):
    return (0.20 * x - 0.26 * y, 0.23 * x + 0.22 * y + 1.6)

def function4(x, y):
    return (-0.15 * x + 0.28 * y, 0.26 * x + 0.24 * y + 0.44)

# Store into list
functions = [function1, function2, function3, function4]

# Image size
WIDTH, HEIGHT = 1920, 1080
DATAPOINTS = 100
x, y = 0, 0
fern_image = np.zeros((HEIGHT, WIDTH, 3))  # Use 3 channels for RGB colors
    
# Set the entire background to black
fern_image[:] = [0, 0, 0]

if not os.path.exists("images/individual") or not os.path.exists("images/3d"):
    os.makedirs("images/individual")
    os.makedirs("images/demo")
    os.makedirs("images/3d/horizontal")
    os.makedirs("images/3d/vertical")
    
fern_3d_pts = []
    
for i in range(DATAPOINTS):
    function = np.random.choice(functions, p=[0.01, 0.85, 0.07, 0.07])
    x, y = function(x, y)
    shift_x, shift_y = int(WIDTH / 2 + x * WIDTH / 10), int(y * HEIGHT / 12)
    
    fern_3d_pts.append((shift_x, shift_y, i))

    # Set each shifted pixel to be blue
    fern_image[shift_y, shift_x] = [0, 0, 255]

    # Normalize the image
    fern_image_normalized = fern_image / 255.0

    # Save the image
    filename = f"images/individual/fern_point_{i:05d}.png"
    plt.imsave(filename, fern_image_normalized, dpi=300)

plt.close()

fern_3d_pts = np.array(fern_3d_pts)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
# ax.grid(False)
# ax.xaxis.pane.fill = False
# ax.yaxis.pane.fill = False
# ax.zaxis.pane.fill = False
# plt.axis('off')

for angle in range(0, 360, 1):
    ax.view_init(elev=0, azim=angle)
    ax.scatter(fern_3d_pts[:, 0], fern_3d_pts[:, 1], fern_3d_pts[:, 2], c='blue', cmap=cm.viridis, s=0.1, alpha=0.5)
    filename = f"images/3d/horizontal/fern_3d_angle_{angle:03d}.png"
    plt.savefig(filename, dpi=300)
    
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
# ax.grid(False)
# ax.xaxis.pane.fill = False
# ax.yaxis.pane.fill = False
# ax.zaxis.pane.fill = False
# plt.axis('off')

for elevation in range(0, 360, 1):
    ax.view_init(elev=elevation, azim=0)
    ax.scatter(fern_3d_pts[:, 0], fern_3d_pts[:, 1], fern_3d_pts[:, 2], c='blue', cmap=cm.viridis, s=0.1, alpha=0.5)
    filename = f"images/3d/vertical/fern_3d_elevation_{elevation:03d}.png"
    plt.savefig(filename, dpi=300)
    
plt.close()
    
"""
Note:
The image frames were stitched together using FFMPEG
Microsoft Clipchamp was used to increase the speed of the video
LumaAI was used to create an interactive 3D model
"""

In [ ]:
import subprocess
# Run ffmpeg command to create a video from the images
subprocess.run(["ffmpeg", "-framerate", "25", "-i", f"images/individual/fern_point_%05d.png", "images/demo/BarnsleyFractal.mp4"])
subprocess.run(["ffmpeg", "-framerate", "25", "-i", f"images/3d/horizontal/fern_3d_angle_%03d.png", "images/demo/angle.mp4"])
subprocess.run(["ffmpeg", "-framerate", "25", "-i", f"images/3d/vertical/fern_3d_elevation_%03d.png", "images/demo/elevation.mp4"])

In [ ]:
angle_video = "images/demo/angle.mp4"
elevation_video = "images/demo/elevation.mp4"
combined_video = "images/demo/combined.mp4"

command = [
    'ffmpeg',
    '-i', 'images/demo/angle.mp4',
    '-i', 'images/demo/elevation.mp4',
    '-filter_complex', '[0:v] [1:v] concat=n=2:v=1:a=0 [v]',
    '-map', '[v]',
    'images/demo/combined.mp4'
]

try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Success!")
except subprocess.CalledProcessError as e:
    print("Error:", e.stderr)